In [1]:
import nibabel as nib
import matplotlib.pyplot as plt

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import torchvision.transforms as transforms
import numpy as np
import torchvision.models as models

In [3]:
# 自定义数据集类
class SpineWeb15(Dataset):
    def __init__(self, data_path, transform=None):
        self.data_path = data_path
        self.transform = transform
        
    def __len__(self):
        return len(self.data_path)
    
    def __getitem__(self, index):
        metadata = np.load(self.data_path+str(index)+".npy")
        image = metadata[0]
        label = metadata[1]

        image_data = torch.from_numpy(image).float()
        label_data = torch.from_numpy(label).float()
        
        if self.transform:
            image_data = self.transform(image_data)
        return image_data, label_data



In [4]:
# VGG Net structure

class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        # 加载预训练的VGG模型作为基础模型
        self.vgg = models.vgg16(pretrained=True)
        # 更改第一层的图像输入维度为1，为灰度图像
        self.vgg.features[0] = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)
        # 替换最后一层全连接层，根据需要调整输出通道数
        self.vgg.classifier[6] = nn.Linear(4096, 128*128)  # 输出通道数为128*128

    def forward(self, x):
        x = self.vgg.features(x)
        x = self.vgg.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.vgg.classifier(x)
        x = x.view(-1, 1, 128, 128)
        return x


In [5]:
# -----------------------------
# VGG16 Net
# -----------------------------

# empty the cache for model and training process
torch.cuda.empty_cache()

# set the hyper parameters and the paths 设置超参数和路径
data_path = "./dataset/Spineweb_dataset15/processed/"

#hyper parameters set
batch_size = 16
num_epochs = 100
learning_rate = 0.0001
in_channels = 1  # according to demand to adjust the number of channels 根据实际情况修改通道数
out_channels = 1  # according to demand to adjust the number of channels 根据实际情况修改通道数

# 创建数据集和数据加载器
transform = transforms.Compose([
    transforms.ToPILImage(),  # 转换为 PIL 图像对象
    transforms.Resize((128, 128)),  # 调整大小为 128x128
    transforms.ToTensor()  # 转换为张量
])
dataset = SpineWeb15(data_path=data_path,transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 创建模型和优化器
model = VGG16()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.BCEWithLogitsLoss()  # 二分类任务可以使用BCEWithLogitsLoss

# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# check the parameters of the model could be trained 检查模型的参数是否设置为可训练
# for name, param in model.named_parameters():
#     print(f"Parameter: {name}, Requires Grad: {param.requires_grad}")

best_loss = 100.0
best_IoU = 0.0
best_Dice = 0.0

# 训练模型
for epoch in range(num_epochs):
    torch.cuda.empty_cache()
    model.train()
    train_loss = 0
    total_iou = 0
    total_dice = 0

    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)
    
        optimizer.zero_grad()

        outputs = model(images)
        # squeeze the dimension that pytorch add for channels
        outputs = outputs.squeeze(1)     # tensor size ([16,128,128])
        # process the result, bigger than 0.5 is reliable
        predicted = (outputs > 0.5).float()     # tensor size ([16,128,128])

        intersection = torch.logical_and(predicted, labels).sum((1, 2))
        union = torch.logical_or(predicted, labels).sum((1, 2))
        iou = (intersection / (union + 1e-7)).mean()  # 平均每个样本的IoU
        dice = (2 * intersection / (predicted.sum((1, 2)) + labels.sum((1, 2)) + 1e-7)).mean()  # 平均每个样本的Dice Coefficient

        loss = criterion(outputs, labels)

        train_loss += loss.item()
        total_iou += iou.item()
        total_dice += dice.item()

        loss.backward()
        optimizer.step()

    avg_loss = train_loss / len(dataloader)
    avg_iou = total_iou / len(dataloader)
    avg_dice = total_dice / len(dataloader)
    
    # get the best IoU and Dice
    best_loss = best_loss if best_loss < avg_loss else avg_loss
    best_IoU = best_IoU if best_IoU > avg_iou else avg_iou
    best_Dice = best_Dice if best_Dice > avg_dice else avg_dice

    print(f"Epoch [{epoch+1}/{num_epochs}], Avg Loss: {avg_loss:.4f}, Avg IoU: {avg_iou:.4f}, Avg Dice: {avg_dice:.4f}")
print("-----------------------------------------")
print(f"Best Loss: {best_loss:.4f}, Best IoU: {best_IoU:.4f}, Best Dice: {best_Dice:.4f}")
# empty the cache for model and training process
torch.cuda.empty_cache()

c:\Users\manjw\.conda\envs\dataEngineering\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\manjw\.conda\envs\dataEngineering\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


OutOfMemoryError: CUDA out of memory. Tried to allocate 392.00 MiB (GPU 0; 4.00 GiB total capacity; 3.08 GiB already allocated; 0 bytes free; 3.12 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# 在测试集上评估模型
model.eval()
test_loss = 0
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_dataloader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        test_loss += criterion(outputs, labels).item()

test_accuracy = 100 * correct / total
print('Test Accuracy: {:.2f}%'.format(test_accuracy))